In [ ]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import tensorflow as tf
#import keras
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate,Activation, Multiply, Dropout, Subtract,MaxPooling2D,ZeroPadding2D,Convolution2D
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
from deepface import DeepFace
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_input_facenet

2021-09-15 01:36:47.273415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
import pandas as pd
import numpy as np
#import seaborn as sns
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from random import choice, sample
import random 
import cv2
from imageio import imread
# from keras.preprocessing.text import Tokenizer, one_hot
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model, load_model
# from keras import regularizers
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization,Dense, concatenate, Flatten, Conv1D
# from keras.optimizers import RMSprop, Adam
# from keras_vggface.vggface import VGGFace
from glob import glob
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D, Lambda, Reshape
from collections import defaultdict
# from keras_vggface.utils import preprocess_input

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.test.is_gpu_available())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3671733584146702053
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14646682624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11999655187236681609
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
True


2021-09-15 01:36:53.855668: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-15 01:36:53.856737: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-15 01:36:53.888092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 01:36:53.888712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-09-15 01:36:53.888740: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-15 01:36:53.891682: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
train_folders_path = "./kinship_dataset/train-faces/"
val_folders_path   = "./kinship_dataset/val-faces/" 

In [ ]:
all_images_train = list(set(glob(train_folders_path + "*/*/*.jpg")))
all_images_val   = list(set(glob(val_folders_path + "*/*/*.jpg")))
print(len(all_images_train))
print(len(all_images_val))

21920
5045


In [ ]:
all_images_train[:2]

['./kinship_dataset/train-faces/F0844/unrelated_and_nonfaces/P08912_face0.jpg',
 './kinship_dataset/train-faces/F0733/MID1/P07684_face0.jpg']

In [ ]:
import random

def process_data(image_path_list):
    #people = set([])
    #person_to_images_map = defaultdict(list)
    family_faces = defaultdict(list)
    #family_size = {}

    for x in image_path_list: 
        splits = x.split("/")

        family = splits[-3]
        # family_size.setdefault(family, 0)
        # family_size[family] += 1

        person = splits[-3] + "/" + splits[-2]
        #people.add(person)

        #person_to_images_map[person].append(x)
        family_faces[family].append(x)

    return family_faces

In [ ]:
# process train data 
family_faces_train = process_data(all_images_train)
families_train = list(family_faces_train.keys())
random.shuffle(families_train)
print(len(families_train))

# process val data 
family_faces_val = process_data(all_images_val)
families_val = list(family_faces_val.keys())
random.shuffle(families_val)
print(len(families_val))

571
192


In [ ]:
family_faces = family_faces_train
family_faces.update(family_faces_val)
family_faces_val = None
family_faces_train = None

In [ ]:
# provided label is not used 

# train_file_path = "./train-pairs.csv"
# relationships_raw = pd.read_csv(train_file_path)
# relationships_raw = list(zip(relationships_raw.fid1.values, relationships_raw.fid2.values))
# print(len(relationships_raw))


Create dataset (mother-father are kin in this)

In [ ]:
#Visualize one family
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# for face in family_faces_train['F0207']:
#   plt.figure()
#   img = mpimg.imread(face)
#   plt.imshow(img)

In [ ]:
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [ ]:
# all persons within the same family are considered kin
from random import choices

def gen(list_tuples, batch_size, read_img):
    while True:
        # get families
        batch_tuples = choices(list_tuples, k=batch_size//2)
        kin_tuples = [(batch_tuples[i], batch_tuples[i]) for i in range(len(batch_tuples))]

        # combine families
        batch_tuples = sample(list_tuples, k=batch_size)
        batch_tuples = [(batch_tuples[i], batch_tuples[i+1]) for i in range(0, len(batch_tuples), 2)]

        # make labels
        labels = [1]*(batch_size//2) + [0]*(batch_size//2)
        batch_tuples = kin_tuples + batch_tuples


        # get people in families
        tuples = []
        for a in batch_tuples:
          notfound = True
          while(notfound):
            p1 = choice(family_faces[a[0]])
            p2 = choice(family_faces[a[1]])
            if "/".join(p1.split("/")[-3:-1]) != "/".join(p2.split("/")[-3:-1]):
              tuples.append((p1, p2))
              notfound = False
        batch_tuples = [(a[0], a[1], b) for (a, b) in zip(tuples, labels)]
        random.shuffle(batch_tuples)
        X1 = [x[0] for x in batch_tuples]
        input_3 = np.array([read_img(x) for x in X1])
        

        X2 = [x[1] for x in batch_tuples]
        input_4 = np.array([read_img(x) for x in X2])
        labels = [x[2] for x in batch_tuples]

        yield [input_3, input_4], np.array(labels)


In [ ]:
def model_1():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    vgg_model = VGGFace(model='resnet50', input_shape=(224, 224, 3), include_top=False, pooling='None')
    out = vgg_model.get_layer('avg_pool').output
    base_model = Model(vgg_model.input, out)

    for layer in base_model.layers:
        layer.trainable = False

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    
    x4  = Subtract()([x1_, x2_])  
    x4 = Lambda(lambda tensor  : K.abs(tensor))(x4)

    x5 = Multiply()([x1, x2])

    x   = Concatenate(axis=-1)([x5, x4, x3])
    x   = BatchNormalization()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x) 
    out = Dense(1, activation="sigmoid")(x)
  

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.001))
    #model.summary()

    return model

In [ ]:
def model_2():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    vgg_model = VGGFace(model='senet50', input_shape=(224, 224, 3), include_top=False, pooling='None')
    out = vgg_model.get_layer('avg_pool').output
    base_model = Model(vgg_model.input, out)

    for layer in base_model.layers:
        layer.trainable = False

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    
    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    
    x4  = Subtract()([x1_, x2_])  
    x4 = Lambda(lambda tensor  : K.abs(tensor))(x4)

    x5 = Multiply()([x1, x2])

    x   = Concatenate(axis=-1)([x5, x4, x3])
    x   = BatchNormalization()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x) 
    out = Dense(1, activation="sigmoid")(x)
  

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.001))
    #model.summary()

    return model

In [ ]:
def model_3():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    vgg_model = VGGFace(model='resnet50', input_shape=(224, 224, 3), include_top=False, pooling='None')
    out = vgg_model.get_layer('avg_pool').output
    base_model = Model(vgg_model.input, out)

    for layer in base_model.layers:
        layer.trainable = False

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    
    x4  = Subtract()([x1_, x2_])  
    x4 = Lambda(lambda tensor  : K.abs(tensor))(x4)

    x5 = Multiply()([x1, x2])

    x   = Concatenate(axis=-1)([x5, x4, x3])
    x   = BatchNormalization()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x) 
    out = Dense(1, activation="sigmoid")(x)
  

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.001))
    #model.summary()

    return model

In [ ]:
def model_4():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    vgg_model = VGGFace(model='senet50', input_shape=(224, 224, 3), include_top=False, pooling='None')
    out = vgg_model.get_layer('avg_pool').output
    base_model = Model(vgg_model.input, out)

    for layer in base_model.layers:
        layer.trainable = False

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_ = Multiply()([x1, x1])
    x2_ = Multiply()([x2, x2])
    
    x4  = Subtract()([x1_, x2_])  
    x4 = Lambda(lambda tensor  : K.abs(tensor))(x4)

    x5 = Multiply()([x1, x2])

    x   = Concatenate(axis=-1)([x5, x4, x3])
    x   = BatchNormalization()(x)
    x   = Dropout(0.5)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(256, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x)
    x   = Dense(128, activation="relu")(x)
    x   = BatchNormalization()(x)
    x   = Dropout(0.3)(x) 
    out = Dense(1, activation="sigmoid")(x)
  

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.001))
    #model.summary()

    return model

In [ ]:
def train(model, model_name, model_max_acc_dict, 
          read_img_for_train, read_img_for_val, learning_rate, early_stop_threshold=10, 
          generator_func=gen, train_relationships=[], val_relationships=[]): 
    print("Training model " + model_name)
    weight_file_path = "./models/" + model_name + ".weights"

    epoch_max = 200
    early_stop_threshold = early_stop_threshold
    reduce_threshold = 3

    early_stop_count = 1
    reduce_count = 1
    val_acc_max = -1
    acc_max = -1

    for epoch_index in range(1, epoch_max): 
        model.fit_generator(generator=generator_func(train_relationships, batch_size=32, read_img=read_img_for_train), 
                  validation_data=generator_func(val_relationships, batch_size=16, read_img=read_img_for_val),
                  epochs=1, 
                  verbose=1, 
                  steps_per_epoch=200,
                  validation_steps=50)
        
        new_val_acc = model.history.history["val_acc"][-1]
        if new_val_acc > val_acc_max: 
            val_acc_max = new_val_acc 
            model_max_acc_dict[model_name] = val_acc_max
            np.save(weight_file_path, np.array(model.get_weights()))
            print("saved weights for model at " + weight_file_path)
            early_stop_count = 1
        else: 
            early_stop_count += 1
        if early_stop_count >= early_stop_threshold: 
            print("end training for model early")
            break 
        
        new_acc = model.history.history["acc"][-1]
        if new_acc > acc_max: 
            acc_max = new_acc
            reduce_count = 1
        else: 
            reduce_count += 1
        if reduce_count >= reduce_threshold: 
            learning_rate = learning_rate * 0.1
            model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(learning_rate))
            print("reduced learning rate to " + str(learning_rate))

    json_object = json.dumps(model_max_acc_dict)
    with open("./models/model_max_acc_dict.json", "w") as outfile:
        outfile.write(json_object)

In [ ]:
import json
from pprint import pprint as pp
model_max_acc_dict = {}
with open('./models/model_max_acc_dict.json', 'r') as openfile:
    model_max_acc_dict = json.load(openfile)
    pp(model_max_acc_dict)

{'model_1.1': 0.6812499761581421,
 'model_1.2': 0.699999988079071,
 'model_1.3': 0.7174999713897705,
 'model_1.4': 0.6962500214576721,
 'model_2.1': 0.6875,
 'model_2.2': 0.7212499976158142,
 'model_2.3': 0.7275000214576721,
 'model_2.4': 0.6899999976158142,
 'model_3.1': 0.7099999785423279,
 'model_3.2': 0.699999988079071,
 'model_3.3': 0.6762499809265137,
 'model_3.4': 0.7024999856948853,
 'model_4.1': 0.7137500047683716,
 'model_4.2': 0.7087500095367432,
 'model_4.3': 0.6912500262260437,
 'model_4.4': 0.668749988079071}


In [ ]:
families =  families_train + families_val
families_train = families_val = None
random.shuffle(families)

In [ ]:
# train the same model multiples times based on different split of train and validation data
import gc

k_fold = 4

size_ = len(families) // k_fold
groups = [[x, x + size_] for x in range(0, len(families), size_)]
groups[-1][1] = len(families)

model_factory = model_1
model_name = "model_1."
model = None

for i in range(0, k_fold): 
    val_start_index, val_end_index = groups[i]
    val_relationships = families[val_start_index:val_end_index]
    train_relationships = families[0:val_start_index] \
                        + families[val_end_index:len(families)]
    del model
    gc.collect()
    model = model_factory()
    train(model, model_name + str(i+1), model_max_acc_dict, 
          read_img, read_img, 0.001, 10, gen, 
          train_relationships, val_relationships)



Training model model_1.1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 63s 213ms/step - loss: 0.8798 - acc: 0.5531 - val_loss: 0.7205 - val_acc: 0.5863


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_1.1.weights
200/200 [==============================] - 40s 201ms/step - loss: 0.6923 - acc: 0.6112 - val_loss: 0.6472 - val_acc: 0.6288
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_1.1.weights
200/200 [==============================] - 39s 195ms/step - loss: 0.6428 - acc: 0.6278 - val_loss: 0.6129 - val_acc: 0.6350
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_1.1.weights
200/200 [==============================] - 40s 199ms/step - loss: 0.6020 - acc: 0.6622 - val_loss: 0.5978 - val_acc: 0.6575
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_1.1.weights
200/200 [==============================] - 39s 194ms/step - loss: 0.5963 - acc: 0.6595 - val_loss: 0.5927 - val_acc: 0.6475
reduced learning rate to 0.0001
200/200 [==============================] - 45s 201ms/step - loss: 0.5919 - acc: 0.6588 - val_loss: 0.5986 - val_acc: 0.6725
saved we

In [ ]:
# train the same model multiples times based on different split of train and validation data
import gc 

k_fold = 4
random.shuffle(families)
size_ = len(families) // k_fold
groups = [[x, x + size_] for x in range(0, len(families), size_)]
groups[-1][1] = len(families)

model_factory = model_2
model_name = "model_2."
model = None

for i in range(0, k_fold): 
    val_start_index, val_end_index = groups[i]
    val_relationships = families[val_start_index:val_end_index]
    train_relationships = families[0:val_start_index] \
                        + families[val_end_index:len(families)]
    del model
    gc.collect()
    model = model_factory()
    train(model, model_name + str(i+1), model_max_acc_dict, 
          read_img, read_img, 0.001, 10, gen, 
          train_relationships, val_relationships)



Training model model_2.1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 69s 225ms/step - loss: 0.8202 - acc: 0.5713 - val_loss: 0.6229 - val_acc: 0.6475


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_2.1.weights
200/200 [==============================] - 38s 192ms/step - loss: 0.6052 - acc: 0.6550 - val_loss: 0.6134 - val_acc: 0.6538
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_2.1.weights
200/200 [==============================] - 38s 191ms/step - loss: 0.5757 - acc: 0.6862 - val_loss: 0.5818 - val_acc: 0.6875
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_2.1.weights
200/200 [==============================] - 38s 190ms/step - loss: 0.5629 - acc: 0.6889 - val_loss: 0.6140 - val_acc: 0.6413
end training for model early
Training model model_2.2
200/200 [==============================] - 48s 201ms/step - loss: 0.8451 - acc: 0.5547 - val_loss: 0.6576 - val_acc: 0.6275
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_2.2.weights
200/200 [==============================] - 39s 195ms/step - loss: 0.6770 - acc: 0.6134 - val_loss: 0.6061 - va

In [ ]:
# train the same model multiples times based on different split of train and validation data
import gc

k_fold = 4

size_ = len(families) // k_fold
groups = [[x, x + size_] for x in range(0, len(families), size_)]
groups[-1][1] = len(families)

model_factory = model_3
model_name = "model_3."
model = None

for i in range(0, k_fold): 
    val_start_index, val_end_index = groups[i]
    val_relationships = families[val_start_index:val_end_index]
    train_relationships = families[0:val_start_index] \
                        + families[val_end_index:len(families)]
    del model
    gc.collect()
    model = model_factory()
    train(model, model_name + str(i+1), model_max_acc_dict, 
          read_img, read_img, 0.001, 10, gen, 
          train_relationships, val_relationships)



94699520/94694792 [==============================] - 1s 0us/step
Training model model_3.1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 2893s 14s/step - loss: 0.8958 - acc: 0.5143 - val_loss: 0.6588 - val_acc: 0.5962


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_3.1.weights
200/200 [==============================] - 1515s 8s/step - loss: 0.7396 - acc: 0.5475 - val_loss: 0.6270 - val_acc: 0.6300
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_3.1.weights
200/200 [==============================] - 381s 2s/step - loss: 0.6377 - acc: 0.6227 - val_loss: 0.6163 - val_acc: 0.6438
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_3.1.weights
200/200 [==============================] - 268s 1s/step - loss: 0.6313 - acc: 0.6319 - val_loss: 0.6024 - val_acc: 0.6637
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_3.1.weights
200/200 [==============================] - 218s 1s/step - loss: 0.6143 - acc: 0.6470 - val_loss: 0.5964 - val_acc: 0.6700
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_3.1.weights
200/200 [==============================] - 165s 829ms/step - loss: 0.6071 - acc: 0.655

In [ ]:
# train the same model multiples times based on different split of train and validation data
import gc

k_fold = 4

size_ = len(families) // k_fold
groups = [[x, x + size_] for x in range(0, len(families), size_)]
groups[-1][1] = len(families)

model_factory = model_4
model_name = "model_4."
model = None

for i in range(0, k_fold): 
    val_start_index, val_end_index = groups[i]
    val_relationships = families[val_start_index:val_end_index]
    train_relationships = families[0:val_start_index] \
                        + families[val_end_index:len(families)]
    del model
    gc.collect()
    model = model_factory()
    train(model, model_name + str(i+1), model_max_acc_dict, 
          read_img, read_img, 0.001, 5, gen, 
          train_relationships, val_relationships)



Training model model_4.1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


200/200 [==============================] - 77s 256ms/step - loss: 0.8737 - acc: 0.5127 - val_loss: 0.6276 - val_acc: 0.6488


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_4.1.weights
200/200 [==============================] - 45s 226ms/step - loss: 0.6392 - acc: 0.6212 - val_loss: 0.5938 - val_acc: 0.6900
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_4.1.weights
200/200 [==============================] - 45s 225ms/step - loss: 0.6130 - acc: 0.6420 - val_loss: 0.5629 - val_acc: 0.7138
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_4.1.weights
200/200 [==============================] - 45s 226ms/step - loss: 0.5905 - acc: 0.6714 - val_loss: 0.5686 - val_acc: 0.7088
end training for model early
Training model model_4.2
200/200 [==============================] - 56s 237ms/step - loss: 0.8952 - acc: 0.5158 - val_loss: 0.6777 - val_acc: 0.5625
saved weights for model at /gdrive/MyDrive/FG 2021 RFIW2021/models/model_4.2.weights
200/200 [==============================] - 46s 228ms/step - loss: 0.7217 - acc: 0.5698 - val_loss: 0.6324 - va

In [ ]:
import json
from pprint import pprint as pp
model_max_acc_dict = {}
with open('./models/model_max_acc_dict.json', 'r') as openfile:
    model_max_acc_dict = json.load(openfile)
    pp(model_max_acc_dict)

{'model_1.1': 0.6812499761581421,
 'model_1.2': 0.699999988079071,
 'model_1.3': 0.7174999713897705,
 'model_1.4': 0.6962500214576721,
 'model_2.1': 0.6875,
 'model_2.2': 0.7212499976158142,
 'model_2.3': 0.7275000214576721,
 'model_2.4': 0.6899999976158142,
 'model_3.1': 0.7099999785423279,
 'model_3.2': 0.699999988079071,
 'model_3.3': 0.6762499809265137,
 'model_3.4': 0.7024999856948853,
 'model_4.1': 0.7137500047683716,
 'model_4.2': 0.7087500095367432,
 'model_4.3': 0.6912500262260437,
 'model_4.4': 0.668749988079071}


In [ ]:
model_func_dict = {
 'model_1.1': model_1,
 'model_1.2': model_1,
 'model_1.3': model_1,
 'model_1.4': model_1,
 'model_2.1': model_2,
 'model_2.2': model_2,
 'model_2.3': model_2,
 'model_2.4': model_2,
 'model_3.1': model_3,
 'model_3.2': model_3,
 'model_3.3': model_3,
 'model_3.4': model_3,
 'model_4.1': model_4,
 'model_4.2': model_4,
 'model_4.3': model_4,
 'model_4.4': model_4,
 }

model_weight_paths = {
 'model_1.1': "./models/model_1.1.weights.npy",
 'model_1.2': "./models/model_1.2.weights.npy",
 'model_1.3': "./models/model_1.3.weights.npy",
 'model_1.4': "./models/model_1.4.weights.npy",
 'model_2.1': "./models/model_2.1.weights.npy",
 'model_2.2': "./models/model_2.2.weights.npy",
 'model_2.3': "./models/model_2.3.weights.npy",
 'model_2.4': "./models/model_2.4.weights.npy",
 'model_3.1': "./models/model_3.1.weights.npy",
 'model_3.2': "./models/model_3.2.weights.npy",
 'model_3.3': "./models/model_3.3.weights.npy",
 'model_3.4': "./models/model_3.4.weights.npy",
 'model_4.1': "./models/model_4.1.weights.npy",
 'model_4.2': "./models/model_4.2.weights.npy",
 'model_4.3': "./models/model_4.3.weights.npy",
 'model_4.4': "./models/model_4.4.weights.npy",
 }

In [ ]:
def get_models(weight_by_acc=True, filter_threshold=1):
    models = []

    if weight_by_acc: 
        for model_name, weight_file_path in model_weight_paths.items(): 
            acc = model_max_acc_dict[model_name]
            print("loading ", model_name, ", val acc is ", acc)
            if acc < filter_threshold: 
                print("\tval acc is lower than ", str(filter_threshold), ", skipping")
                continue

            model = model_func_dict[model_name]()
            with open(weight_file_path, 'rb') as f:
                weights = np.load(f, allow_pickle=True)
                model.set_weights(weights) 
                models.append((model, acc))

    else: # weight each model equally
        for model_name, weight_file_path in model_weight_paths.items(): 
            print("loading ", model_name, " with equal weight")
            acc = model_max_acc_dict[model_name]
            if acc < filter_threshold: 
                print("\tval acc is lower than ", str(filter_threshold), ", skipping")

            model = model_func_dict[model_name]()
            with open(weight_file_path, 'rb') as f:
                weights = np.load(f, allow_pickle=True)
                model.set_weights(weights) 
                models.append((model, 1))

    return models

In [ ]:
models = get_models(weight_by_acc=False, filter_threshold=0)
# normalize weights of models
sum_acc = sum([x[1] for x in models])
models = [(x[0], x[1]/sum_acc) for x in models]
pp(models)

loading  model_1.1  with equal weight


2021-09-15 01:38:26.507674: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-15 01:38:26.507950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-15 01:38:26.508663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-09-15 01:38:26.508710: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-15 01:38:26.508771: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-15 01:38:26.508786: I tensorflow/stream_executor/platform/default/dso_lo

loading  model_1.2  with equal weight
loading  model_1.3  with equal weight
loading  model_1.4  with equal weight
loading  model_2.1  with equal weight
104947712/104944616 [==============================] - 2s 0us/step
loading  model_2.2  with equal weight
loading  model_2.3  with equal weight
loading  model_2.4  with equal weight
loading  model_3.1  with equal weight
loading  model_3.2  with equal weight
loading  model_3.3  with equal weight
loading  model_3.4  with equal weight
loading  model_4.1  with equal weight
loading  model_4.2  with equal weight
loading  model_4.3  with equal weight
loading  model_4.4  with equal weight
[(<tensorflow.python.keras.engine.functional.Functional object at 0x7f53c40ca070>,
  0.0625),
 (<tensorflow.python.keras.engine.functional.Functional object at 0x7f53c053d9a0>,
  0.0625),
 (<tensorflow.python.keras.engine.functional.Functional object at 0x7f53c02133d0>,
  0.0625),
 (<tensorflow.python.keras.engine.functional.Functional object at 0x7f53c0603550>

In [ ]:
# produce predictions on test set for submission

# Modify paths as per your need

test_path = "./test-faces/"
submission = pd.read_csv('./test-full.csv')
predictions = []

for i in range(0, len(submission.p1_new.values), 32):
    print(i)
    X1 = submission.p1_new.values[i:i+32]
    X2 = submission.p2_new.values[i:i+32]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models: 
        #pred = model.predict([input_1, input_2], verbose=0) * weight
        pred1 = model.predict([input_1, input_2], verbose=0) 
        pred2 = model.predict([input_1_2, input_2_2], verbose=0) 
        pred3 = model.predict([input_1_3, input_2_3], verbose=0)
        pred = (pred1 + pred2 + pred3) * weight / 3
        preds.append(pred)
    preds = np.array(preds).sum(axis=0).squeeze().tolist()
    
    predictions.extend(preds)


In [ ]:
d = {'index': np.arange(0, len(predictions), 1), 'label':predictions}
submissionfile  = pd.DataFrame(data=d)
print(submissionfile["label"].min(), submissionfile["label"].max(), submissionfile["label"].mean(), submissionfile["label"].median())

0.00943293608725071 0.9998732209205627 0.49000495099264435 0.47689127922058105


In [ ]:
temp = (submissionfile["label"] > submissionfile["label"].median()).astype(int)
#temp = (submissionfile["label"] > 0.5).astype(int)
print("acc", ((temp[:19890]).sum() + len(submissionfile) -19889 - (temp[19890:]).sum())/len(submissionfile))

acc 0.7667010542737086


In [ ]:
submissionfile["label"] = (submissionfile["label"] > submissionfile["label"].median()).astype(int)
#submissionfile["label"] = (submissionfile["label"] > 0.5).astype(int)

In [ ]:
submissionfile.astype('int64').to_csv("./predictions.csv", index=False)

In [ ]:
# produce predictions on test set for submission

# Modify paths as per your need

test_path = "./kinship_dataset/test-faces/"
submission = pd.read_csv('./kinship_dataset/test-full.csv')
predictions_ = {}

batch_size = 64
for i in range(0, len(submission.p1_new.values), batch_size):
    print(i)
    X1 = submission.p1_new.values[i:i+batch_size]
    X2 = submission.p2_new.values[i:i+batch_size]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models: 
        pred1 = model([input_1, input_2], training=False) 
        pred2 = model([input_1_2, input_2_2], training=False) 
        pred3 = model([input_1_3, input_2_3], training=False) 
        pred = (pred1 + pred2 + pred3) * weight / 3
        preds.append(pred)

    preds = (np.array(preds).sum(axis=0).squeeze() > 0.47689127922058105).astype(int)
    labels = submission.labels[i:i+batch_size]
    right_or_wrong = (preds == np.array(labels)).tolist()

    ptypes = submission.ptype[i:i+batch_size].tolist()
    for j in range(len(ptypes)): 
        ptype = ptypes[j]
        predictions_.setdefault(ptype, {"count": 0, "correct": 0})
        predictions_[ptype]["correct"] += right_or_wrong[j]
        predictions_[ptype]["count"] += 1
    


In [ ]:
for ptype in predictions_: 
  print(ptype, predictions_[ptype]["correct"] * 1.0/ predictions_[ptype]["count"])

bb 0.8291413703382481
ss 0.8301759133964818
fd 0.7164624047697913
md 0.7650753768844221
fs 0.7613809960281087
gmgd 0.29577464788732394
gmgs 0.3333333333333333
ms 0.7229323308270676
gfgs 0.53125
gfgd 0.4628099173553719
sibs 0.7776628748707343
nan 0.766886616632247


In [ ]:
# ensemble of model_1

test_path = "./kinship_dataset/test-faces/"
submission = pd.read_csv('./kinship_dataset/test-full.csv')
predictions_ = {}

batch_size = 64
for i in range(0, len(submission.p1_new.values), batch_size):
    print(i)
    X1 = submission.p1_new.values[i:i+batch_size]
    X2 = submission.p2_new.values[i:i+batch_size]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models[0:4]: 
        pred1 = model([input_1, input_2], training=False) 
        pred2 = model([input_1_2, input_2_2], training=False) 
        pred3 = model([input_1_3, input_2_3], training=False) 
        pred = (pred1 + pred2 + pred3) * weight * 4 / 3
        preds.append(pred)

    preds = (np.array(preds).sum(axis=0).squeeze() > 0.47689127922058105).astype(int)
    labels = submission.labels[i:i+batch_size]
    right_or_wrong = (preds == np.array(labels)).tolist()

    ptypes = submission.ptype[i:i+batch_size].tolist()
    for j in range(len(ptypes)): 
        ptype = ptypes[j]
        predictions_.setdefault(ptype, {"count": 0, "correct": 0})
        predictions_[ptype]["correct"] += right_or_wrong[j]
        predictions_[ptype]["count"] += 1
    
for ptype in predictions_: 
    print(ptype, predictions_[ptype]["correct"] * 1.0/ predictions_[ptype]["count"])

0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10368
10432
10496
10560
10624
10688
10752
10816
10880
10944
11008
11072
11136
11200
11264
11328
11392
11456
11520
11584
11648
11712
11776
11840
11904
11968
12032
12096
12160
12224
12288
12352
12416
12480


In [ ]:
correct_sum = 0
count_sum = 0
for ptype in predictions_: 
    correct_sum += predictions_[ptype]["correct"] 
    count_sum += predictions_[ptype]["count"]
print(correct_sum * 1.0 / count_sum)

0.74991822459301


In [ ]:
# ensemble of model_2

test_path = "./kinship_dataset/test-faces/"
submission = pd.read_csv('./kinship_dataset/test-full.csv')
predictions_ = {}

batch_size = 64
for i in range(0, len(submission.p1_new.values), batch_size):
    print(i)
    X1 = submission.p1_new.values[i:i+batch_size]
    X2 = submission.p2_new.values[i:i+batch_size]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models[4:8]: 
        pred1 = model([input_1, input_2], training=False) 
        pred2 = model([input_1_2, input_2_2], training=False) 
        pred3 = model([input_1_3, input_2_3], training=False) 
        pred = (pred1 + pred2 + pred3) * weight * 4 / 3
        preds.append(pred)

    preds = (np.array(preds).sum(axis=0).squeeze() > 0.47689127922058105).astype(int)
    labels = submission.labels[i:i+batch_size]
    right_or_wrong = (preds == np.array(labels)).tolist()

    ptypes = submission.ptype[i:i+batch_size].tolist()
    for j in range(len(ptypes)): 
        ptype = ptypes[j]
        predictions_.setdefault(ptype, {"count": 0, "correct": 0})
        predictions_[ptype]["correct"] += right_or_wrong[j]
        predictions_[ptype]["count"] += 1
    
correct_sum = 0
count_sum = 0    
for ptype in predictions_: 
    print(ptype, predictions_[ptype]["correct"] * 1.0/ predictions_[ptype]["count"])
    correct_sum += predictions_[ptype]["correct"] 
    count_sum += predictions_[ptype]["count"]
print(correct_sum * 1.0 / count_sum)  

0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10368
10432
10496
10560
10624
10688
10752
10816
10880
10944
11008
11072
11136
11200
11264
11328
11392
11456
11520
11584
11648
11712
11776
11840
11904
11968
12032
12096
12160
12224
12288
12352
12416
12480


In [ ]:
# ensemble of model_3

test_path = "./kinship_dataset/test-faces/"
submission = pd.read_csv('./kinship_dataset/test-full.csv')
predictions_ = {}

batch_size = 64
for i in range(0, len(submission.p1_new.values), batch_size):
    print(i)
    X1 = submission.p1_new.values[i:i+batch_size]
    X2 = submission.p2_new.values[i:i+batch_size]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models[8:12]: 
        pred1 = model([input_1, input_2], training=False) 
        pred2 = model([input_1_2, input_2_2], training=False) 
        pred3 = model([input_1_3, input_2_3], training=False) 
        pred = (pred1 + pred2 + pred3) * weight * 4 / 3
        preds.append(pred)

    preds = (np.array(preds).sum(axis=0).squeeze() > 0.47689127922058105).astype(int)
    labels = submission.labels[i:i+batch_size]
    right_or_wrong = (preds == np.array(labels)).tolist()

    ptypes = submission.ptype[i:i+batch_size].tolist()
    for j in range(len(ptypes)): 
        ptype = ptypes[j]
        predictions_.setdefault(ptype, {"count": 0, "correct": 0})
        predictions_[ptype]["correct"] += right_or_wrong[j]
        predictions_[ptype]["count"] += 1
    
correct_sum = 0
count_sum = 0    
for ptype in predictions_: 
    print(ptype, predictions_[ptype]["correct"] * 1.0/ predictions_[ptype]["count"])
    correct_sum += predictions_[ptype]["correct"] 
    count_sum += predictions_[ptype]["count"]
print(correct_sum * 1.0 / count_sum)  

0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10368
10432
10496
10560
10624
10688
10752
10816
10880
10944
11008
11072
11136
11200
11264
11328
11392
11456
11520
11584
11648
11712
11776
11840
11904
11968
12032
12096
12160
12224
12288
12352
12416
12480


In [ ]:
# ensemble of model_4

test_path = "./kinship_dataset/test-faces/"
submission = pd.read_csv('./kinship_dataset/test-full.csv')
predictions_ = {}

batch_size = 64
for i in range(0, len(submission.p1_new.values), batch_size):
    print(i)
    X1 = submission.p1_new.values[i:i+batch_size]
    X2 = submission.p2_new.values[i:i+batch_size]

    input_1 = np.array([read_img(test_path + x) for x in X1])
    input_2 = np.array([read_img(test_path + x) for x in X2])

    input_1_2 = np.array([np.flip(x, axis=1) for x in input_1])
    input_2_2 = np.array([np.flip(x, axis=1) for x in input_2])

    input_1_3 = np.array([np.flip(x, axis=-1) for x in input_1])
    input_2_3 = np.array([np.flip(x, axis=-1) for x in input_2])

    preds = []
    for (model, weight) in models[12:16]: 
        pred1 = model([input_1, input_2], training=False) 
        pred2 = model([input_1_2, input_2_2], training=False) 
        pred3 = model([input_1_3, input_2_3], training=False) 
        pred = (pred1 + pred2 + pred3) * weight * 4 / 3
        preds.append(pred)

    preds = (np.array(preds).sum(axis=0).squeeze() > 0.47689127922058105).astype(int)
    labels = submission.labels[i:i+batch_size]
    right_or_wrong = (preds == np.array(labels)).tolist()

    ptypes = submission.ptype[i:i+batch_size].tolist()
    for j in range(len(ptypes)): 
        ptype = ptypes[j]
        predictions_.setdefault(ptype, {"count": 0, "correct": 0})
        predictions_[ptype]["correct"] += right_or_wrong[j]
        predictions_[ptype]["count"] += 1
    
correct_sum = 0
count_sum = 0    
for ptype in predictions_: 
    print(ptype, predictions_[ptype]["correct"] * 1.0/ predictions_[ptype]["count"])
    correct_sum += predictions_[ptype]["correct"] 
    count_sum += predictions_[ptype]["count"]
print(correct_sum * 1.0 / count_sum)  

0
64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10368
10432
10496
10560
10624
10688
10752
10816
10880
10944
11008
11072
11136
11200
11264
11328
11392
11456
11520
11584
11648
11712
11776
11840
11904
11968
12032
12096
12160
12224
12288
12352
12416
12480
